# CLASE 1.2: OPERATORIA EN NUMPY
---
## Una observación acerca de los tiempos de ejecución
Hasta ahora, hemos estado discutiendo algunos de los conceptos más elementales de **Numpy**; en las próximas secciones de este módulo, navegaremos entre las razones por las cuales **Numpy** es tan importante en el mundo de la ciencia de datos con Python. A saber, nos provee de una interfaz sencilla y flexible para la computación optimizada con arreglos de datos.

Los cálculos con arreglos de **Numpy** pueden ser extremadamente rápidos, o bien, muy muy lentos. La clave para hacerlos rápidos yace en la utilización de **operaciones vectorizadas** que, en general, se implementan mediante las llamadas funciones universales de **Numpy**, conocidas en la práctica como `ufuncs`. Procederemos a motivar la necesidad por el uso de las `ufuncs`, las cuales pueden ser utilizadas para la realización de cálculos recursivos sobre los elementos de un arreglo de manera mucho más eficiente. A partir de dicha motivación, introduciremos muchas de las `ufuncs` más comunes de **Numpy**.

La implementación estándar de Python, conocida como *CPython*, realiza algunas operaciones muy lentamente. Esto es, en parte, debido a la naturaleza interpretativa y dinámica de Python. La flexibilidad de los tipos provoca que Python infiera nuestros datos sin necesidad de que los declaremos, pero también hace que dichos datos tengan mayor complejidad, lo que implica que, cuando hacemos cálculos recursivos con ellos (por ejemplo, en bucles), esa lentitud se manifieste de manera significativa. Por ejemplo:

In [1]:
import numpy as np

In [2]:
# Fijamos un generador de números aleatorios.
rng = np.random.default_rng(seed=7)

In [3]:
# Construimos una función que nos permite calcular las potencias negativas de 2 conforme los 
# exponentes que se listan en un arreglo.
def compute_neg_exp(values):
    
    # Inicializamos el output de nuestro cálculo mediante un arreglo vacío (función np.empty()).
    output = np.empty(len(values))
    
    # Hacemos el cálculo mediante un bucle sencillo.
    for i in range(len(values)):
        output[i] = 2**(-values[i])
    
    return output

In [4]:
# Definimos un arreglo unidimensional para almacenar los exponentes que queremos evaluar.
values = rng.normal(loc=5, scale=2, size=5)

La implementación anterior no parece tener ningún problema. Pero si medimos el **tiempo de ejecución** de este código para un input muy grande, veremos que esta operación se ejecuta de manera muy lenta… ¡Quizás demasiado lenta! Mediremos dicho tiempo de ejecución mediante el uso de un código mágico de **IPython** denominado `%timeit`:

In [5]:
# Construimos un arreglo de un millón de filas para testear el tiempo de ejecución de nuestra función.
huge_array = rng.normal(loc=5, scale=2, size=1000000)

In [6]:
# Medimos el tiempo de ejecución de la funcion que construimos sobre este arreglo.
%timeit compute_neg_exp(huge_array)

279 ms ± 7.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


¡Toma varios milisegundos calcular estos millones de operaciones y almacenar el resultado! En un mundo en el cual incluso los celulares tienen velocidades de procesamiento de información medidas en Giga-FLOPS (es decir, miles de millones de cálculos por segundo), esto parece ser ridículamente lento. Sucede que el cuello de botella aquí no son las operaciones propiamente tales, sino que el chequeo de los tipos de datos subyacentes y los despachos de funciones que realiza CPython en cada ciclo del bucle. Cada vez que un recíproco se calcula, Python primero examina el tipo de dato que caracteriza al objeto respectivo y realiza un chequeo dinámico de la función correcta que debe usarse para ese tipo de dato.

Es por eso que, a fin de ser capaces de asegurar que nuestros cálculos, basados en arreglos de **Numpy**, tengan un tiempo de ejecución razonable, tenemos que conocer cómo operar con tales arreglos mediante operaciones especialmente construidas para trabajar con ellos.

## Funciones universales.
Para muchos tipos de operaciones, **Numpy** nos provee de una conveniente interfaz en este tipo de rutinas, de manera similar a lo que se lograría mediante un código ya compilado. Aquello se conoce como **operación vectorizada**. Podemos lograr esto simplemente realizando una operación sobre el arreglo, la cual luego será aplicada a cada elemento del mismo. Este enfoque vectorizado está diseñado para forzar a cualquier bucle a trabajar más rápido aprovechando las bondades de **Numpy**, logrando así una ejecución más efectiva.

Comparemos los tiempos de ejecución de la operación definida por nuestra función `compute_neg_exp()`, que trabaja mediante un bucle, y la misma operación trabajada por esta función, aplicada directamente sobre nuestro arreglo gigante (`huge_array`):

In [7]:
%timeit 1/2**(-huge_array)

8.56 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Vemos que la operación anterior, aplicada directamente sobre el arreglo `huge_array`, se ejecuta muchísimo más rápido que en el caso de la función `compute_neg_exp()`, ya que ahora el cálculo completo toma 10.6 milisegundos, lo que es casi 38 veces más rápido que el primer cálculo que realizamos.

Las operaciones vectorizadas es **Numpy** se implementan mediante el uso de funciones universales (comúnmente abreviadas como `ufuncs`, del inglés *universal functions*), cuyo gran propósito es ejecutar rápidamente operaciones recursivas sobre los elementos que conforman un arreglo de **Numpy**. Las ufuncs son extremadamente flexibles; recién vimos una operación entre un arreglo y un escalar, pero también podemos operar sobre dos arreglos:

In [8]:
np.linspace(start=1, stop=10, num=10) / np.linspace(start=11, stop=20, num=10)

array([0.09090909, 0.16666667, 0.23076923, 0.28571429, 0.33333333,
       0.375     , 0.41176471, 0.44444444, 0.47368421, 0.5       ])

El código anterior muestra la operación aritmética de división entre arreglos. Sin embargo, antes de proceder, es buena idea revisar cómo se opera aritméticamente con estas estructuras de datos, a fin de tener muy claro, en términos matemáticos, qué es lo que estamos haciendo. Esto es importante, puesto que partimos este estudio hablando de cómo los arreglos pueden ser utilizados para representar objetos matemáticos típicos del álgebra lineal, tales como vectores, matrices y tensores. Y si bien hay operaciones tales como la suma de arreglos, que es exactamente la misma suma definida sobre tales objetos matemáticos, otras operaciones no tienen una equivalencia necesariamente obvia.

### Aritmética de arreglos.
En el contexto del álgebra lineal, existe un tipo de estructura de gran importancia conocida como **espacio vectorial**, la cual puede crearse a partir de un conjunto no vacío (cuyos elementos son llamados vectores), una operación interna (llamada suma, definida para los elementos del conjunto) y una operación externa (llamada producto por un escalar, definida entre dicho conjunto y otro conjunto, con estructura de cuerpo, como podrían ser los conjuntos $\mathbb{R}$ o $\mathbb{C}$) que satisface varias propiedades fundamentales. Tales propiedades se resumen en un conjunto de reglas que permiten darle sentido a las operaciones previamente mencionadas.

En la asignatura de álgebra lineal, aprendimos que tanto los vectores en $\mathbb{R}^n$ como las matrices (que se representan como $\mathbb{R}^{m\times n}$, para el caso de matrices con $m$ filas y $n$ columnas) conforman una estructura de espacio vectorial. La operación interna que define dicha estructura en ambos casos corresponde a la suma (de vectores o matrices). Sean $\mathbf{a}=(a_{1},...,a_{m})$ y $\mathbf{b}=(b_{1},...,b_{m})$ dos vectores en $\mathbb{R}^{m}$. La suma de estos vectores es otro vector, que denotamos como $\mathbf{a}+\mathbf{b}$, y que se define como

<p style="text-align: center;">$\mathbf{a}+\mathbf{b}=(a_{1}+b_{1},...,a_{m}+b_{m})\in \mathbb{R}^{m}$</p>

Es decir, para sumar dos vectores, estos deben ser de la misma dimensión, y la suma resultará en otro vector cuyos elementos serán iguales a la suma de los elementos de cada uno de los vectores originales, posición a posición. Esto también se cumple para el caso de las matrices en ℝ^(𝑚×𝑛); es decir, si $\mathbf{A} =\left\{a_{ij}\right\}$ y $\mathbf{B} =\left\{b_{ij}\right\}$ son matrices en $\mathbb{R}^{m\times n}$, la suma $\mathbf{A}+\mathbf{B}$ se definirá como

<p style="text-align: center;">$\mathbf{A} +\mathbf{B} =\left\{ a_{ij}+b_{ij}\right\}  =\left( \begin{matrix}a_{11}+b_{11}&\cdots &a_{1n}+b_{1n}\\ \vdots &\ddots &\vdots \\ a_{m1}+b_{m1}&\cdots &a_{mn}+b_{mn}\end{matrix} \right)  \in \mathbb{R}^{m\times n} $</p>

La suma así definida es exactamente la misma suma que opera sobre arreglos en **Numpy**. Por ejemplo, si definimos los arreglos:

In [9]:
# Dos arreglos unidimensionales (similes a vectores).
a = np.array([0, -1, 8, -2, 5])
b = np.array([-6, -9, 0, 1, -2])

In [10]:
# Dos arreglos bidimensionales (similares a matrices).
A = np.array([
    [-1, 0, 1, 5, 2, 6],
    [-4, 0, 0, 4, -1, 0],
    [-8, -1, 3, 0, -2, 1],
    [0, -1, -1, 4, -2, 1],
])
B = np.array([
    [-2, -2, 0, 6, 1, -1],
    [0, -1, 8, -6, -8, 0],
    [9, 1, -7, 6, 1, -9],
    [0, -4, 4, 0, 0, -5],
])

In [11]:
# La suma a + b resulta en un arreglo también 1D, cuyos elementos son las sumas de los 
# correspondientes elementos de a y b.
a + b

array([ -6, -10,   8,  -1,   3])

In [12]:
# La suma A + B resulta en un arreglo también 2D, cuyos elementos son las sumas de los 
# correspondientes elementos de A y B.
A + B

array([[-3, -2,  1, 11,  3,  5],
       [-4, -1,  8, -2, -9,  0],
       [ 1,  0, -4,  6, -1, -8],
       [ 0, -5,  3,  4, -2, -4]])

La resta, que en términos algebraicos no es más que un caso particular de la suma, también es una operación interna que trabaja de la misma forma sobre los correspondientes arreglos:

In [13]:
# La resta a - b.
a - b

array([ 6,  8,  8, -3,  7])

In [14]:
# La resta A - B.
A - B

array([[  1,   2,   1,  -1,   1,   7],
       [ -4,   1,  -8,  10,   7,   0],
       [-17,  -2,  10,  -6,  -3,  10],
       [  0,   3,  -5,   4,  -2,   6]])

Veamos ahora el caso del producto por un escalar. 

En estricto rigor, un escalar no es más que un valor que pertenece ya sea a $\mathbb{R}$ o $\mathbb{C}$, según sea el caso. En términos de **Numpy**, los escalares son simplemente los números que pueden constituir los elementos de un arreglo respectivo. Desde la perspectiva algebraica, el producto de un vector por un escalar puede definirse como sigue: Sea $\mathbf{a}=(a_{1},...,a_{m})\in \mathbb{R}$ un vector y $\lambda \in \mathbb{R}$ un escalar. Entonces el producto del vector $\mathbf{a}$ por el escalar $\lambda$ es también un vector, denotado como $\lambda \mathbf{a}$, y que se define como

<p style="text-align: center;">$\lambda \mathbf{a}=(\lambda a_{1},...,\lambda a_{m})\in \mathbb{R}$</p>

Esta operación se define de manera completamente análoga para el caso de las matrices. Es decir, para la matriz $\mathbf{A} =\left\{ a_{ij}\right\}  \in \mathbb{R}^{m\times n} $, el producto entre el escalar $\lambda$ y dicha matriz, denotado como $\lambda \mathbf{A}$, es también una matriz, y se calcula como

<p style="text-align: center;">$\lambda \mathbf{A} =\left\{ \lambda a_{ij}\right\}  =\left( \begin{matrix}\lambda a_{11}&\cdots &\lambda a_{1n}\\ \vdots &\ddots &\vdots \\ \lambda a_{m1}&\cdots &\lambda a_{mn}\end{matrix} \right)  \in \mathbb{R}^{m\times n}$</p>

Para el caso de los arreglos de **Numpy**, la replicación de esta operación externa es igual de sencilla que en el caso de la suma:

In [15]:
lam = 2 # Un escalar con un valor igual a 2.

In [16]:
# Multiplicación de un arreglo unidimensional por este escalar.
lam * a

array([ 0, -2, 16, -4, 10])

In [17]:
# Multiplicación de un arreglo bidimensional por este escalar.
lam * A

array([[ -2,   0,   2,  10,   4,  12],
       [ -8,   0,   0,   8,  -2,   0],
       [-16,  -2,   6,   0,  -4,   2],
       [  0,  -2,  -2,   8,  -4,   2]])

Vemos pues que las operaciones de suma y producto por un escalar siguen exactamente la misma lógica cuando representamos vectores y matrices mediante arreglos unidimensionales y bidimensionales, respectivamente.

Sin embargo, existen otras operaciones que no resultan obvias en su ejecución cuando homologamos este tipo de objetos matemáticos a arreglos, y es muy buena idea detenernos a explicarlas a fin de evitar cualquier tipo de confusión. En **Numpy**, las cuatro operaciones básicas de la aritmética (digamos, suma, resta, multiplicación y división) siguen la misma lógica que la suma (vista como una operación interna) definida previamente. Es decir, siempre operan componente a componente. De esta manera, para el caso los arreglos bidimensionales `A` y `B` que construimos un par de láminas atrás, la multiplicación `A * B` también da como resultado un arreglo, ya que:

In [18]:
# La multiplicación directa de arreglos bidimensionales en Numpy trabaja como una operación interna: Componente a componente.
A * B

array([[  2,   0,   0,  30,   2,  -6],
       [  0,   0,   0, -24,   8,   0],
       [-72,  -1, -21,   0,  -2,  -9],
       [  0,   4,  -4,   0,   0,  -5]])

Esto también se cumple para el caso de los arreglos unidimensionales:

In [19]:
# La multiplicación directa de arreglos unidimensionales en Numpy también es componente a componente.
a * b

array([  0,   9,   0,  -2, -10])

La división también trabaja exactamente igual:

In [20]:
# La división de arreglos bidimensionales también trabaja componente a componente.
A / B

/var/folders/sn/hlgjzv2n1537dplx8bvvj1c80000gn/T/ipykernel_31421/3816306364.py:2: RuntimeWarning: divide by zero encountered in true_divide
  A / B
/var/folders/sn/hlgjzv2n1537dplx8bvvj1c80000gn/T/ipykernel_31421/3816306364.py:2: RuntimeWarning: invalid value encountered in true_divide
  A / B


array([[ 0.5       , -0.        ,         inf,  0.83333333,  2.        ,
        -6.        ],
       [       -inf, -0.        ,  0.        , -0.66666667,  0.125     ,
                nan],
       [-0.88888889, -1.        , -0.42857143,  0.        , -2.        ,
        -0.11111111],
       [        nan,  0.25      , -0.25      ,         inf,        -inf,
        -0.2       ]])

In [21]:
# Mismo caso para los arreglos unidimensionales.
a / b

/var/folders/sn/hlgjzv2n1537dplx8bvvj1c80000gn/T/ipykernel_31421/3266095820.py:2: RuntimeWarning: divide by zero encountered in true_divide
  a / b


array([-0.        ,  0.11111111,         inf, -2.        , -2.5       ])

Los arreglos resultantes de una división como la anterior pueden tener elementos indeterminados (`nan`) o con una magnitud demasiado grande (`inf`). Esto es porque, como cabría esperar, la división así definida requiere que los elementos del arreglo divisor sean todos distintos de cero.

Es justo preguntarnos si esta forma de multiplicar existe formalmente en el área del álgebra lineal. Y resulta que, en efecto, esta operación sí está bien definida. Para todo par de matrices $\mathbf{A} =\left\{ a_{ij}\right\}$ y $\mathbf{B} =\left\{b_{ij}\right\}$ en $\mathbb{R}^{m\times n}$, se define el **producto de Hadamard**, denotado como $\mathbf{A} \odot \mathbf{B}$, como la matriz cuyos elementos son las multiplicaciones, posición a posición, de los elementos originales de las matrices $\mathbf{A}$ y $\mathbf{B}$. Es decir,

<p style="text-align: center;">$\mathbf{A} \odot \mathbf{B} =\left\{ a_{ij}b_{ij}\right\}  =\left( \begin{matrix}a_{11}b_{11}&\cdots &a_{1n}b_{1n}\\ \vdots &\ddots &\vdots \\ a_{m1}b_{m1}&\cdots &a_{mn}b_{mn}\end{matrix} \right)  \in \mathbb{R}^{m\times n} $</p>

Por lo tanto, la operación de multiplicación `A * B` que está implementada en **Numpy** para los arreglos `A` y `B` es, indudablemente, el producto de Hadamard. De hecho, la división `A / B` es un caso particular de este producto, donde la multiplicación se aplica sobre los recíprocos de la matriz `B`. Si volvemos al contexto del álgebra lineal, podemos observar que el producto de matrices es muy diferente del producto de Hadamard. Al respecto, sean las matrices $\mathbf{A} =\left\{ a_{ij}\right\}\in \mathbb{R}^{m\times n}$ y $\mathbf{B} =\left\{ b_{ij}\right\}\in \mathbb{R}^{m\times k}$. Los elementos $\left\{ c_{ij}\right\} $ de la matriz producto $\mathbf{A} \mathbf{B} \in \mathbb{R}^{m\times k}$ se definen mediante la fórmula siguiente:

<p style="text-align: center;">$c_{ij}={\displaystyle \sum^{n}_{s=1} a_{is}b_{sj}}\  ;\  i=1,...,m\  \wedge \  j=1,...,k$</p>

Por lo tanto, para computar el elemento $c_{ij}$ de la matriz $\mathbf{C}=\mathbf{A} \mathbf{B}$, multiplicamos los elementos de la i-ésima fila de $\mathbf{A}$ con la j-ésima columna de $\mathbf{B}$, y luego los sumamos. Esto pone de manifiesto que las matrices $\mathbf{A}$ y $\mathbf{B}$ deben cumplir con ser compatibles para la multiplicación matricial así definida. Por ejemplo, una matriz $\mathbf{A}$ de $n\times k$ puede ser multiplicada por otra matriz $\mathbf{B}$ de $k\times m$, pero solamente de izquierda a derecha:

<p style="text-align: center;">$\underbrace{\mathbf{A} }_{n\times k} \  \underbrace{\mathbf{B} }_{k\times m} =\underbrace{\mathbf{C} }_{n\times m}$</p>

El producto $\mathbf{B} \mathbf{A}$ no está definido si $n\neq m$, ya que las dimensiones respectivas no son compatibles.

Este producto también puede implementarse en **Numpy**. La función universal (o `ufunc`) encargada de hacerlo es `np.matmul()`, la cual toma como argumentos dos arreglos que nos interesa multiplicar, siempre que éstos sean compatibles para esta operación. Ejemplifiquemos esto mediante la definición de dos nuevas matrices `A` y `B`, definidas como

In [22]:
# Definimos los arreglos A y B.
A = np.array([
    [1, 2, 3],
    [3, 2, 1]
])
B = np.array([
    [0, 2],
    [1, -1],
    [0, 1]
])

In [23]:
# La multiplicación matricial de ambos arreglos se realiza mediante la ufunc np.matmul().
np.matmul(A, B)

array([[2, 3],
       [2, 5]])

In [24]:
# En este caso, ambos arreglos son compartibles para ser multiplicados en ambos sentidos.
np.matmul(B, A)

array([[ 6,  4,  2],
       [-2,  0,  2],
       [ 3,  2,  1]])

La multiplicación matricial, como podemos observar, no es una operación conmutativa. Por lo tanto, no cumple con ser una operación interna válida para poder construir un espacio vectorial. En **Numpy**, el operador rápido para la implementación de este producto corresponde a `@`. De esta manera, en vez de utilizar la función `np.matmul()`, podemos simplemente escribir:

In [25]:
# El operador @ permite multiplicar matricialmente nuestros arreglos.
A @ B

array([[2, 3],
       [2, 5]])

In [26]:
B @ A

array([[ 6,  4,  2],
       [-2,  0,  2],
       [ 3,  2,  1]])

Volvamos a la operación de producto y consideremos ahora arreglos unidimensionales. Debido a que éstos pueden ser usados para representar vectores, es evidente que el producto definido en primera instancia (`a * b`, en nuestro ejemplo) es también un producto de Hadamard (componente a componente). Sin embargo, como en el caso de las matrices, es probable que recordemos que el concepto de producto visto en nuestros cursos de Cálculo (e incluso de Física) es muy diferente.

Partamos con el concepto de **producto interno** (o producto punto o escalar) de vectores. Para todo par de vectores $a, b\in \mathbb{R}^{n}$, tales que $\mathbf{a}=(a_{1},...,a_{n})$ y $\mathbf{b}=(b_{1},...,b_{n})$, se define el producto interno $\mathbf{a} \cdot \mathbf{b} $ como

<p style="text-align: center;">$\mathbf{a} \cdot \mathbf{b} =a_{1}b_{1}+\cdots +a_{n}b_{n}={\displaystyle \sum^{n}_{i=1} a_{i}b_{i}}\in \mathbb{R}$</p>

Es decir, el producto interno $\mathbf{a} \cdot \mathbf{b} $ de los vectores $\mathbf{a}$ y $\mathbf{b}$ da como resultado un escalar.

La implementación de este producto interno en **Numpy** se realiza por medio de la función `np.dot()`. De esta manera, para el caso de los arreglos unidimensionales a y b definidos previamente, tendremos que su producto interno puede calcularse como

In [27]:
# El producto interno entre los arreglos unidimensionales a y b se calcula de la siguiente manera.
np.dot(a, b)

-3

Notemos que, en efecto, el resultado anterior es un escalar.

Si los argumentos de la función `np.dot()` son arreglos bidimensionales, el resultado de la misma será el producto matricial equivalente al que obtendríamos por medio del uso de de la función `np.matmul()`. Esto podemos chequearlo rápidamente mediante la función `np.allclose()`, la cual evalúa el resultado de dos operaciones y devuelve el valor Booleano `True` cuando tales resultados son iguales:

In [28]:
# np.dot() y np.matmul() son equivalentes cuando usamos arreglos bidimensionales.
np.allclose(np.dot(A, B), np.matmul(A, B))

True

Ahondaremos más en operaciones propias del álgebra lineal más adelante.

Resumiendo las consideraciones previamente establecidas, las `ufuncs` de **Numpy** se sienten muy naturales de utilizar debido a que hacen uso de los operadores aritméticos nativos de Python. Las operaciones aritméticas de adición, sustracción, multiplicación y división, por tanto, pueden expresarse como sigue:

In [29]:
# Construimos el arreglo x.
x = np.array([-1, 0, 9, 5, -4, 5])

In [30]:
# Las operaciones aritméticas en Numpy son, en resumen, muy intuitivas.
print("x =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)

x = [-1  0  9  5 -4  5]
x + 5 = [ 4  5 14 10  1 10]
x - 5 = [-6 -5  4  0 -9  0]
x * 2 = [-2  0 18 10 -8 10]
x / 2 = [-0.5  0.   4.5  2.5 -2.   2.5]
x // 2 = [-1  0  4  2 -2  2]


También existe una `ufunc` unaria para la negación, un operador `**` para la exponenciación, y un operador `%` para el cálculo de restos de divisiones (módulos):

In [31]:
# Operaciones de negación, exponenciación y módulo.
print("-x =", -x)
print("x ** 2 =", x ** 2)
print("x % 2 =", x % 2)

-x = [ 1  0 -9 -5  4 -5]
x ** 2 = [ 1  0 81 25 16 25]
x % 2 = [1 0 1 1 0 1]


En adición, éstas `ufuncs` pueden mezclarse como sea que deseemos, teniendo en consideración siempre que las operaciones respetarán la jerarquía de las operaciones aritméticas:

In [32]:
-(0.5*x + 1) ** 2

array([ -0.25,  -1.  , -30.25, -12.25,  -1.  , -12.25])

### Valor absoluto y norma.
Estamos familiarizados con el concepto de valor absoluto, el cual, para el caso de los números reales, corresponde a la distancia absoluta a la cual éstos se emplazan en la recta real respecto del origen de dicha recta (el número 0). Por lo tanto, para todo número real $x$ se define la función valor absoluto como

<p style="text-align: center;">$f\left( x\right)  =\left| x\right|  =\begin{cases}x&;\  \mathrm{s} \mathrm{i} \  x\geq 0\\ -x&;\  \mathrm{s} \mathrm{i} \  x<0\end{cases} $</p>

El concepto de valor absoluto es extensible a los elementos de un arreglo siempre que opere únicamente sobre los elementos del mismo, posición a posición. De esta manera, en **Numpy**, el valor absoluto puede implementarse mediante la función `np.abs()`, la cual permite calcular dicho valor absoluto para todos los elementos de un arreglo:

In [33]:
# Construimos un arreglo bidimensional A.
A = np.array([
    [-1, -6, 0, 7, 8],
    [5, -3, -4, 8, -9],
    [-6, 6, 7, 1, 0],
    [9, -1, -1, 0, -7]
])

In [34]:
# Calculamos el valor absoluto de sus elementos.
np.abs(A)

array([[1, 6, 0, 7, 8],
       [5, 3, 4, 8, 9],
       [6, 6, 7, 1, 0],
       [9, 1, 1, 0, 7]])

Si los elementos de un arreglo son números complejos (conforme la Tabla (1.1), el tipo `np.complex64` o `np.complex128`), la función `np.abs()` calculará los módulos asociados a tales elementos. Recordemos que si $z=a+bi\in \mathbb{C}$, con $a,b\in \mathbb{R}$, entonces el módulo de $z$ se define como

<p style="text-align: center;">$\left| z\right|={\displaystyle \sqrt{a^{2}+b^{2}}}$</p>

Luego, en **Numpy**, tenemos:

In [35]:
# Definimos un arreglo compuesto por números complejos.
Z = np.array([
    [2-3j, 4+1j, -5-6j, -2j],
    [3, -5j, 1-1j, 2-9j],
    [9+2j, -1-8j, 2+3j, -1j]
])

In [36]:
# Calculamos el valor absoluto de los elementos del arreglo Z.
np.abs(Z)

array([[3.60555128, 4.12310563, 7.81024968, 2.        ],
       [3.        , 5.        , 1.41421356, 9.21954446],
       [9.21954446, 8.06225775, 3.60555128, 1.        ]])

El concepto de valor absoluto suele ser asociado a otros resultados más generales que son propios del álgebra lineal o de la física. En particular, cuando pensamos en vectores desde una perspectiva puramente geométrica; es decir, líneas dirigidas que parten desde el origen y terminan en un punto determinado, siempre hemos asociado a estos vectores el concepto de longitud del mismo en términos de la distancia entre el origen y dicho punto. La norma es un concepto que permite formalizar esa noción, siendo dependiente, por supuesto, del tipo de geometría subyacente a la referencia propia del vector. Es muy común trabajar con la norma Euclidiana (o norma $\mathcal{l}_{2}$) de un vector en $\mathbf{x}\in \mathbb{R}^{n}$, la que nos permite calcular la distancia de este vector al origen del sistema de coordenadas que describe su posición:

<p style="text-align: center;">$\left\Vert \mathbf{x} \right\Vert_{2}  :=\sqrt{\displaystyle \sum^{n}_{i=1} x^{2}_{i}} =\sqrt{\mathbf{x}^{\top } \mathbf{x}}$</p>

Si definimos el vector $\mathbf{x}=( 1,-1,4,-8,0,-2)^{\intercal}\in \mathbb{R}^{6}$, veremos que su norma Euclidiana será

<p style="text-align: center;">$\left\Vert \mathbf{x} \right\Vert_{2}  =\sqrt{1^{2}+1^{2}+4^{2}+8^{2}+0^{2}+2^{2}} =\sqrt{86} \approx 9.273$</p>

En **Numpy**, podemos calcular esta norma como sigue:

In [37]:
# Definimos el arreglo 1D (vector) x.
x = np.array([1, -1, 4, -8, 0, -2])

# Calculamos su norma Euclidiana.
np.linalg.norm(x)

9.273618495495704

No ahondaremos mucho más en temas relativos al álgebra lineal en **Numpy** por ahora, ya que esto lo revisaremos un poco más en detalle más adelante.

## Funciones trigonométricas.
**Numpy** nos provee de una enorme cantidad de funciones trascendentes típicas en el análisis de datos. Un ejemplo son las funciones trigonométricas, las cuales son fácilmente representables en **Numpy** mediante las funciones `np.sin()`, `np.cos()` y `np.tan()` para el caso de las funciones seno, coseno y tangente, respectivamente. Para ejemplificar su implementación, definiremos primeramente un arreglo llamado `theta`, que representará ciertos ángulos (en radianes) en términos de ciertos factores de $\pi$. Tal constante puede, igualmente, representarse en **Numpy** mediante `np.pi`. De este modo,

In [38]:
# Definimos un arreglo cuyos elementos serán algunos ángulos (en radianes).
theta = np.linspace(0, np.pi, 4)

In [39]:
# Calculamos los valores de cada una de las funciones trigonométricas para los ángulos que componen nuestro arreglo.
print("theta = ", theta)
print("sen(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta =  [0.         1.04719755 2.0943951  3.14159265]
sen(theta) =  [0.00000000e+00 8.66025404e-01 8.66025404e-01 1.22464680e-16]
cos(theta) =  [ 1.   0.5 -0.5 -1. ]
tan(theta) =  [ 0.00000000e+00  1.73205081e+00 -1.73205081e+00 -1.22464680e-16]


Estos valores se calculan a partir de la precisión numérica que tienen los distintos elementos de un arreglo, considerando el tipo de dato al mismo (atributo `dtype`), razón por la cual, en el bloque de código anterior, observamos valores que deberían ser cero no son exactamente cero.

Las funciones trigonométricas inversas también están disponibles en **Numpy**:

In [40]:
# Funciones trigonométricas inversas.
x = [-1,  -1/2,  0, 1/2, 1]
print("x = ", x)
print("arcsen(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x =  [-1, -0.5, 0, 0.5, 1]
arcsen(x) =  [-1.57079633 -0.52359878  0.          0.52359878  1.57079633]
arccos(x) =  [3.14159265 2.0943951  1.57079633 1.04719755 0.        ]
arctan(x) =  [-0.78539816 -0.46364761  0.          0.46364761  0.78539816]


Las funciones anteriores asumen que los arreglos imputados a las mismas contienen ángulos expresados en radianes, o bien, en el caso de las funciones trigonométricas inversas, retornan arreglos cuyos elementos también son ángulos medidos en radianes. Para hacer conversiones de ángulos a otras unidades de medición, **Numpy** nos provee de algunas funciones muy útiles. Por ejemplo, la función `np.rad2deg()` permite convertir los elementos de un arreglo de radianes a grados (suponiendo, por supuesto, que dicho arreglo almacene ángulos):

In [41]:
# El arreglo theta expresado en grados.
np.rad2deg(theta)

array([  0.,  60., 120., 180.])

La función `np.deg2rad()` permite realizar la conversión inversa, de grados a radianes.

## Funciones logarítmicas y exponenciales.
**Numpy** también nos provee de una serie de implementaciones relativas a funciones exponenciales y logarítmicas. Algunos ejemplos se muestran a continuación:

In [42]:
# Ejemplos de funciones exponenciales (y potenciales).
x = [1, 2, 3, 4, 5]
print("x = ", x)
print("exp(x) = ", np.exp(x))
print("2^x = ", np.exp2(x))
print("3^x = ", np.power(3, x))

x =  [1, 2, 3, 4, 5]
exp(x) =  [  2.71828183   7.3890561   20.08553692  54.59815003 148.4131591 ]
2^x =  [ 2.  4.  8. 16. 32.]
3^x =  [  3   9  27  81 243]


In [43]:
# Ejemplos de funciones logarítmicas.
print("ln(x) = ", np.log(x))
print("log2(x) = ", np.log2(x))
print("log10(x) = ", np.log10(x))

ln(x) =  [0.         0.69314718 1.09861229 1.38629436 1.60943791]
log2(x) =  [0.         1.         1.5849625  2.         2.32192809]
log10(x) =  [0.         0.30103    0.47712125 0.60205999 0.69897   ]


También existen versiones especializadas de funciones logarítmicas y exponenciales que son útiles para mantener la precisión para inputs de magnitudes pequeñas:

In [44]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 = ", np.expm1(x))
print("log(1 + x) = ", np.log1p(x))

exp(x) - 1 =  [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) =  [0.         0.0009995  0.00995033 0.09531018]


Finalmente, **Numpy** dispone de una función que permite calcular raíces cuadradas sin ningún problema:

In [45]:
# La función np.sqrt() permite calcular la raíz cuadrada de los elementos de un arreglo.
x = [1, 4, 9, 16, 25, 36]
print("np.sqrt(x) = ", np.sqrt(x))

np.sqrt(x) =  [1. 2. 3. 4. 5. 6.]


Y eso es todo lo relativo a funciones trascendentes. **Numpy** posee muchísimas otras funciones especializadas, las que pueden ser revisadas en la correspondiente documentación de la librería, junto con los correspondientes ejemplos.

## Comentarios finales.
En esta segunda sección hemos aprendido a operar con arreglos de **Numpy**, considerando la inclusión de funciones algebraicas y trascendentes, incluyendo también elementos importantes del álgebra lineal.

Hasta ahora, ya disponemos de herramientas que nos permiten manipular datos almacenados en arreglos de **Numpy**. Sin embargo, existen ciertas limitaciones en la forma de operar, las que se resumen en que estamos trabajando siempre elemento a elemento, o bien, de forma global en un arreglo. En la siguiente sección procederemos a revisar el concepto de función de agregación, y que nos permitirá construir operaciones que diferencien tanto filas como columnas (y apilamientos) en un arreglo, lo que robustecerá muchísimo más nuestra caja de herramientas para el análisis de datos almacenados en **Numpy**.